# Modelling a perfectly spherical star in python


# Libraries

In [1]:
import math
import decimal #The sun is roughly ~10^27 m^3, making it safe to say the volumes of gases we'll be dealing with will be around ~10^24 m^3, which is far beyond what python's float data type can handle.
import re
import random
from operator import truediv
import hashlib
import numpy as np
import pandas as pd #< To work with mostIsotopesMolarMasses.csv
import pygame
import sys
import logging
import itertools
logging.basicConfig(level=logging.INFO)
from debugpy.launcher.winapi import kernel32

pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Useful constants
Makes life easier.

In [2]:
N_a = 6.02214076 * 10 ** (23)
mu = 1.66054*10**(-27)
e: float = math.e
G: float = 6.6743*10**(-11)
pi: float = math.pi
c: float = 299792458
h: float = 6.6260*10**(-34)
sigma: float = 5.6703*10**(-8)
k: float = 1.38 * 10**(-23)
R: float = 8.3145
epsilon_0: float = 8.854*10**(-12)
e_charge: float = 1.602*10**(-19)
r_0: float = 1.4*10**(-15)

# Simulation time interval

The collapse of gas clouds into stars typically takes millions of years. Unfortunately my computer isn't a supercomputer, and we don't have all year to wait for the results, so we have to take some shortcuts.

We can take much larger time intervals at the beginning of the simulation, where changes in the system are much smaller, and errors will accumulate less. Of course, different sizes and masses of gas clouds will take different times to collapse, so using the free fall equation would tell us what kind of time scale we are looking at.

$$t_{ff} = \sqrt{\frac{3\pi}{32G \rho}}$$

In [3]:
SIMULATION_TIME_INTERVAL = 1
time = 0

In [4]:
totalReactions = 0

# Getting the information out of reactionsInfo.txt

reactionsInfo.txt holds the relevant data for what Reaclib1 file holds the data for which reaction, as well as what the reactants and products are.

The output at the bottom line should look like: 71c7209dc0db7b35cb1201198606aed2

If so, most likely there is a problem in reactionsInfo.txt.

In [5]:
f = open("reactionsInfo.txt", "r")

df = pd.read_csv("C:/Users/dsyao/PycharmProjects/compPhysStars/models/molar_masses/mostIsotopesMolarMasses.csv")

reactionTable = f.read().splitlines()
reactionTable.pop(0)
logging.info(reactionTable)
logging.info("----------------------------------------------------------------------")
reactionTable = [row.split(",") for row in reactionTable]
logging.info(reactionTable)
logging.info("----------------------------------------------------------------------")
reactionTable = [[[row[0]]] + [row[1].split("/")] + [row[2].split("/")] for row in reactionTable]
logging.info(reactionTable)
logging.info(reactionTable[0])
f.close()
hashString = f"{reactionTable}"
hexdigest = hashlib.md5(hashString.encode("utf-8")).hexdigest()
logging.info(hexdigest)
if hexdigest != "71c7209dc0db7b35cb1201198606aed2":
    logging.warning("Erroneous reaction hash")

INFO:root:['ar36-ag-ca40-ths8,1-4-2/1-36-18,1-40-20,', 'b8-a-he4-wc12,1-8-5,1-4-2,', 'be7--li7-ec,1-7-4,1-7-3,', 'be7-pg-b8-nacr,1-7-4/1-1-1,1-8-5,', 'be9-an-c12-cf88,1-9-4/1-4-2,1-12-6/1-1-0,', 'c12-ag-o16-nac2,1-12-6/1-4-2,1-16-8,', 'c12-pg-n13-ls09,1-12-6/1-1-1,1-13-7,', 'c13-pg-n14-nacr,1-13-6/1-1-1,1-14-7,', 'ca40-ag-ti44-chw0,1-40-20/1-4-2,1-44-22,', 'd-pg-he3-de04,1-2-1/1-1-1,1-3-2,', 'f17--o17-wc12,1-17-9,1-17-8,', 'f18--o18-wc12,1-18-9,1-18-8,', 'f19-pa-o16-nacr,1-19-9,1-16-8/1-1-1,', 'he3-ag-be7-cd08,1-3-2/1-4-2,1-7-4,', 'he3-he3pp-he4-nacr,1-3-2/1-3-2,1-4-2/2-1-1,', 'he3-p-he4-bet,1-3-2/1-1-1,1-4-2,', 'he4-aag-c12-fy05,1-4-2,1-12-6,', 'he4-nag-be9-ac12,1-4-2/1-9-4,1-12-6,', 'li7-pa-he4-de04,1-7-3,1-4-2/1-3-1,', 'mg24-ag-si28-st08,1-24-12/1-4-2,1-28-14,', 'n13--c13-wc12,1-13-7,1-13-6,', 'n14-pg-o15-im05,1-14-7/1-1-1,1-15-8,', 'n15-pa-c12-nacr,1-15-7,1-12-6/1-1-1,', 'n15-pg-o16-li10,1-15-7/1-1-1,1-16-8,', 'ne20-ag-mg24-il10,1-20-10/1-4-2,1-24-12,', 'o15--n15-wc12,1-15-8,1-15-7

# Reading Reaclib1 files

In [6]:
#This code was not written by me. It was written by Deepseek

def parse_reaclib1_file(filename):
    """
    Parses a Reaclib1 file and extracts the reaction details and coefficients.
    """
    reactions = []
    with open(filename, 'r') as file:
        lines = file.readlines()

        # Skip the header (chapter number)
        i = 1  # Start reading from the second line

        while i < len(lines):
            # First line of the set entry
            line1 = lines[i].strip()
            if not line1:  # Skip empty lines
                i += 1
                continue

            # Extract reactants and products
            reactants = [line1[5:10].strip(), line1[10:15].strip(), line1[15:20].strip()]
            products = [line1[20:25].strip(), line1[25:30].strip(), line1[30:35].strip()]

            # Extract set label, rate type, reverse flag, and Q value
            set_label = line1[43:47].strip() if len(line1) >= 47 else ''
            rate_type = line1[47] if len(line1) >= 48 else ''
            reverse_flag = line1[48] if len(line1) >= 49 else ''
            q_value = float(line1[52:64]) if len(line1) >= 64 else 0.0

            # Second line of the set entry (first four coefficients)
            line2 = lines[i + 1].strip() if i + 1 < len(lines) else ''
            # Split line2 into coefficients using 'e+' or 'e-' as delimiters
            coeffs_line2 = []
            if line2:
                # Use a regular expression to split the line into valid scientific notation strings
                import re
                parts = re.findall(r'[-+]?\d*\.\d+[eE][-+]?\d+', line2)
                coeffs_line2 = [float(part) for part in parts]

            # Third line of the set entry (last three coefficients)
            line3 = lines[i + 2].strip() if i + 2 < len(lines) else ''
            # Split line3 into coefficients using 'e+' or 'e-' as delimiters
            coeffs_line3 = []
            if line3:
                # Use a regular expression to split the line into valid scientific notation strings
                import re
                parts = re.findall(r'[-+]?\d*\.\d+[eE][-+]?\d+', line3)
                coeffs_line3 = [float(part) for part in parts]

            # Combine all coefficients
            coefficients = coeffs_line2 + coeffs_line3
            if len(coefficients) < 7:
                coefficients.extend([0.0] * (7 - len(coefficients)))  # Pad with zeros if necessary

            # Store the reaction details and coefficients
            reaction = {
                'reactants': reactants,
                'products': products,
                'set_label': set_label,
                'rate_type': rate_type,
                'reverse_flag': reverse_flag,
                'q_value': q_value,
                'coefficients': coefficients
            }
            reactions.append(reaction)

            # Move to the next set entry
            i += 3

    return reactions


def calculate_reaction_rate(T9, coefficients):
    """
    Calculates the reaction rate at a given temperature T9 (in 10^9 K).
    """
    a0, a1, a2, a3, a4, a5, a6 = coefficients
    log_rate = a0 + a1 / T9 + a2 / T9 ** (1 / 3) + a3 * T9 ** (1 / 3) + a4 * T9 + a5 * T9 ** (5 / 3) + a6 * np.log(T9)
    return np.exp(log_rate)

In [7]:
reactionsDetailList = []
for i in reactionTable:
    ee11 = str(i[0][0])

    reactions = parse_reaclib1_file("models/rates/" + ee11)

    coefficientsList = []
    for reaction in reactions:
        coefficientsList.append(reaction['coefficients'])

    logging.info(coefficientsList)


    i.append(coefficientsList)
logging.info("")
for i in reactionTable:
    logging.info(f"{i}\n")

INFO:root:[[52.3486, 0.0, -71.0046, 4.0656, -5.26509, 0.683546, -0.666667]]
INFO:root:[[-0.105148, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
INFO:root:[[-23.8328, 0.0, 0.0, 3.02033, -0.0742132, -0.00792386, -0.650113]]
INFO:root:[[7.73399, -7.345, 0.0, 0.0, 0.0, 0.0, -1.5], [12.5315, 0.0, -10.264, -0.203472, 0.121083, -0.00700063, -0.666667]]
INFO:root:[[11.744, -4.179, 0.0, 0.0, 0.0, 0.0, -1.5], [-1.48281, -1.834, 0.0, 0.0, 0.0, 0.0, -1.5], [-9.51959, -1.184, 0.0, 0.0, 0.0, 0.0, -1.5], [31.464, 0.0, -23.87, 0.566698, 44.0957, -314.232, -0.666667], [19.2962, -12.732, 0.0, 0.0, 0.0, 0.0, 0.0]]
INFO:root:[[254.634, -1.84097, 103.411, -420.567, 64.0874, -12.4624, 137.303], [69.6526, -1.39254, 58.9128, -148.273, 9.08324, -0.541041, 70.3554]]
INFO:root:[[17.1482, 0.0, -13.692, -0.230881, 4.44362, -3.15898, -0.666667], [17.5428, -3.77849, -5.10735, -2.24111, 0.148883, 0.0, -1.5]]
INFO:root:[[13.9637, -5.78147, 0.0, -0.196703, 0.142126, -0.0238912, -1.5], [15.1825, -13.5543, 0.0, 0.0, 0.0, 0.0, -1.5], [

# Defining elements

In [8]:
class element:
    #Proton number
    Z: int
    #Nucleon number
    N: int

    def __eq__(self, other):
        return isinstance(other, element) and self.Z == other.Z and self.N == other.N

    def __hash__(self):
        return hash((self.Z, self.N))

    def __init__(self, Z: int, N: int):
        self.Z = Z
        self.N = N


# The star model

The star will be modelled as a sphere, with many layers of differing compsosition. Each layer will consist of several element, at uniform average temperature. By the nuclear fusion probability function, the amount of that layer that fuses will be calculated, and the new materials will be mixed into the layer. Heavier elements will move towards the core. Descending layers will also take on a larger thickness as the radius grows smaller.

The mass of the star will be calculated by summing the masses of all the layers, as well as adding the mass from the kinetic energy of the molecules.

The star will lose mass via radiation and particles escaping via evaporation.

We can find the power output of the star by using the Stefan-Boltzmann law, assuming the star is a perfect black body.
$$P = \sigma A T^4$$

Evaporation can also be found by finding the proportion of particles at the surface which have sufficient energy to escape the gravitiational field of the sun. This is modelled by using the Maxwell-Boltzmann distribution to find the proportion of particles at the surface of the sun with sufficient energy to escape.

Stars form by clouds of gas collapsing in on themselves, and the gravitational potential energy being converted into kinetic energy. The high temperatures and pressures then drive fusion, which increases the temperature of the gases, increasing pressure and thereby pushing outwards and keeping the star in equilibrium. I will also assume that plasma acts as an ideal gas (even though it doesn't).

$$pV = nRT$$
$$kE = \frac{3}{2} kT$$

As layer n falls, its GPE is converted into KE, where r_0 is the radius between the center of the star and the bottom of the given layer and r_1, the top.

$$\Delta Ke = \frac{2GM_n \sum^{i}_{i=n-1}M_i}{r_{n0} + r_{n1}} - \frac{2GM_n \sum^{i}_{i=n-1}M_i}{r_{n0} + r_{n1} - \Delta r}$$

We can say that $\frac{r_{n0} + r_{n1}}{2} = r$, where R is the total radius of the star. So:

$$\Delta Ke = \frac{GM_n \sum^{i}_{i=n-1}M_i}{r} - \frac{2GM_n \sum^{i}_{i=n-1}M_i}{r - \Delta r}$$

## Forces on a Gas Layer

We model the changes in the layers of the star by measuring the change in the upper bound of each layer, then fixing that value to be the bottom layer of the layer above. The forces acting on any one layer are: gravity, pressure from within the layer, pressure from upper layer, radiation pressure from upper layers, radiation pressure from lower layers.

For layer $n$ or $N$

$$F_g = \frac{GM_n \sum^{i}_{i=n-1}M_i}{r^2}$$

However, there is also a resistive force from the pressure of the upper layer (including radiation pressure), where $\epsilon_j$ is the fraction of radiative pressure left over after passing through layer $j$.

$$F_p = P_{n+1}A + \sum^{N}_{i=n+1} \prod^{N}_{j=n+1} \epsilon_j P_{ri}A = 4 P_{n-1} \pi r_{n0}^2 + P_{r} r_{n0}^2$$

$$F_{total} = F_g - F_{pu} +F_{pl} = \frac{GM_n \sum^{n-1}_{i=1}M_i}{r^2} - 4 P_{n-1} \pi r_{n0}^2 + \sum^{n-1}_{i=1} \prod^{n-1}_{j=1} \epsilon_j P_{ri}A$$


## Cumulative error and motion

Even given our extremely simplified model, the equations describing the layers making up our star model would be much too complicated for me to do. Given the above euqation for force on a layer, the algorithm will calculate how far the layer will move, assuming the force is constant over the simulation time. This is assuming $\Delta r << R$, such that $F_g(t+\Delta t) \approx F_g(t)$ and that $F_p(t+\Delta t) \approx F_p(t)$. However, this method leads to errors, as in a real system, some forces would change as the system evolves, but we do not take that into account in our simulation.

## Adiabatic Heating

One of the main mechanisms by which gases heat up in a star is via adiabatic heating. This is the effect by which compressing a gas causes it to heat up. The equations defining it are:

$$\gamma = \frac{f + 2}{f}$$

Where because all the particles in our star are monatomic, $f = 3$

$$\gamma = \frac{5}{3}$$

And,

$$T_2 = T_1(\frac{P_2}{P_1})^{\frac{\gamma - 1}{\gamma}}$$
$$T_2 = T_1(\frac{P_2}{P_1})^{\frac{2}{5}}$$

## Radiation pressure and radiative heat transfer

The intensity of radiation after passing through a gas of opacity $k$ and thickness $x$ is:

$$I = I_0 e^{-kx \rho}$$

Since I am not smart enough to understand the formulas needed to estimate opacity, I will assume that since most stars are mostly hydrogen, the opactiy will be $k = 0.4$.

$$P = IA$$

$$P = \sigma A T^4$$

$$\Delta T = \frac{2 \Delta E N_a}{3 R}$$

$$E = mc^2$$

$$E = hf$$

$$p = mc = \frac{hf}{c} = \frac{E}{c}$$

$$F = \frac{d p}{d t}$$

$$d p = \frac{P d t}{c}$$

$$F = \frac{\Delta P}{c} = \frac{P(1- e^{-kx \rho})}{c}$$

$$T = T_0 + \frac{2 P \Delta t N_a}{3 R}$$

## Fusion

The REACLIB database returns reaction rates in the form:

$$\lambda = N_a \langle\sigma v\rangle$$

And to use this, we will use the formulas:

For reaction $A + B$

$$r = n_a n_b \langle \sigma v \rangle$$

And for reaction $A + A$

$$r = 0.5 n_a^2 \langle \sigma v \rangle$$

And finally

$$\Delta M_i = r N_a m_i \Delta t$$

Where $m_i$ is the molar mass of $I$

## Heat Transfer

In stars, heat is transferred through it via convection, as well as radiative heating. However, for the first one, our model isn't exacly the best to try and model this, however it might be worth trying to simulate it using conduction between layers. The second major mode of heat transfer is radative heating, which will also cause radiation pressure.

## Gravitational Settling and Convection

In stars heavier elements sink, and lighter elements rise. This is why the core mainly consists of iron, since the heavier elements sink down into the core. However, convection opposes this, meaning (at least for lighter elements), the whole star has some of each element. Unfortunately my model doesn't exactly easily allow for convection, but there are some things we can try.

### Gravitational Settling

For gases, the drift velocity of particle $i$ in a mixture is given by:

$$v_s = \frac{\Delta m g}{k_b T}D$$

Where:

$$\Delta m = m_i - \overline m$$

For species a and b:

$$D \approx \frac{3}{16}\frac{(k_bT)^{\frac{3}{2}}}{p \sigma_{ab}^2\sqrt{2 \pi \mu}}$$

Where:

$$\sigma_{ab}^2 = \pi (r_a + r_b)^2$$

$$r_i \approx 1.2 * 10^{-17}N_i^{\frac{1}{3}}$$

But for multiple isotopes:

$$\frac{1}{D_j} \approx \sum_{i \neq j} \frac{x_i}{D_{ij}}$$

# Order of calculations

1. Calculate forces on each layer
2. Calculate motion of each layer

    a. Calculate motion of upper bound for all layers

    b. Fix lower bound for all layers as upper bound of lower layer.

3. Calculate adiabatic heating

    a. Store previous pressure

    b. Calculate new pressure value

    c. Calculate new temperature
4. Calculate heat transfer between layers
5. Calculate separation of heavy elements between layers
6. Calculate energy radiated away, as well as the spectrum
7. Calculate change in temperature and composition by fusion

This is not perfect, since in reality all these would be happening at the same time. This will also probably lead to some violations of the first law of thermodynamics.

In [9]:
class starLayer:
    #                                                       \/ Mass in Kg     \/ Temp in °K      \/ Radius in m                    \/ Velocity in ms^-1
    def __init__(self, composition: list[tuple[element, decimal.Decimal]], temperature: float, radius_0: float, radius_1: float, velocity: float):
        self.composition = composition
        self.temperature = temperature #We're assuming temperature is homogenous within layers for simplicity's sake
        self.radius_0 = radius_0#We're also assuming density is even within layers
        self.radius_1 = radius_1
        self.velocity = velocity
        self.projectedVelocity = 0
        self.forces = 0
 #       temp = 0
  #      for i in self.composition:
   #         temp += i[1]
        self.mass = np.sum([mass for _, mass in self.composition]) #needs to be recalculated at the beginning of every time interval
        self.pressure = 0.0 #needs to be recalculated at the beginning of every time interval
        self.elementSet = {(e.Z, e.N) for e, _ in self.composition} #needs to be recalculated at the beginning of every time interval
        self.volume = (4/3)*pi*(self.radius_1**3-self.radius_0**3) #needs to be recalculated at the beginning of every time interval
        self.numberOfMoles = 0 #needs to be recalculated at the beginning of every time interval

    def startProcedure(self):
        self.recalculateVolume()
        self.recalculateElementSet()
        self.calculateTotalMass()
        self.calculateNumberOfMoles()
    #--STUFF THAT NEEDS TO BE RECALCULATED AT THE BEGINNING OF EVERY TIME INTERVAL -------

    def recalculateElementSet(self):
        self.elementSet = {(e.Z, e.N) for e, _ in self.composition}

    def calculateNumberOfMoles(self): #Untested
        self.numberOfMoles = sum(tup[1]/df.loc[(df["Z"] == tup[0].Z) & (df["N"] == tup[0].N), "MM"].values for tup in self.composition) * 1000

    def calculateTotalMass(self): #Untested
        self.mass = sum(tup[1] for tup in self.composition)

    def recalculateVolume(self):
        self.volume = (4/3)*pi*(self.radius_1**3-self.radius_0**3)

    def calcAdibetHeat(self): #Untested
        oldPressure = self.pressure
        self.temperature = self.temperature*math.pow(((self.calculatePressure())/(oldPressure)), 0.4)
#self.temperature = self.temperature*math.pow(((self.calculatePressure())/(oldPressure)), 0.4)
    def changeRadii(self, newRadius0, newRadius1): #Untested
        self.radius_0 = newRadius0
        self.radius_1 = newRadius1
    #--SPECIAL RUN CONDITIONS----------------------------------------------------------------

    def calculatePressure(self): #Tested ONLY TO BE RUN BY THE calculateAdibetHeat function
        moles = self.numberOfMoles
        self.pressure = (moles * R * self.temperature)/self.volume
        return self.pressure



    def fusion(self): #Untested
        #random.shuffle(reactionTable) #This is to prevent the reactions to be calculated in the same order every time. If this happens, then it is likely that reactions near the start of the list are overly represented.
        for i in reactionTable:
            self.elementSet = {(e.Z, e.N) for e, _ in self.composition}
            setetete = self.composition
            if self.checkPresence(i) == True:
                #print("found!")

                #print(i[0])
                rate = 0.0

                for z in i[3]:
                    # \/ Rate is in cm^3 mol^-1 s^-1, we want cm^3 s^-1
                    #ratesList = parse_reaclib1_file(f"C:/Users/dsyao/PycharmProjects/compPhysStars/models/rates/{i[0][0]}")

                    #for j in ratesList:
                    rate += calculate_reaction_rate(T9 = self.temperature/1000000000, coefficients=z)
                    #logging.info(rate)

                #print(f"volume: {self.volume}")
                #print(f"rate: {rate}")

                rate = rate/(N_a * 1000000)

                if len(i[1]) == 2:
                    massDefct = 0.0
                    temp3 = i[1][0].split("-")
                    temp4 = i[1][1].split("-")
                    el1 = element(Z = int(temp3[2]), N = int(temp3[1]))
                    massDefct += df.loc[(df["Z"] == int(temp3[2])) & (df["N"] == int(temp3[1])), "MM"].values[0] * int(i[1][0][0])
                    mass1 = next((mass for e, mass in self.composition if e.Z == el1.Z and e.N == el1.N), 0)
                    el2 = element(Z = int(temp4[2]), N = int(temp4[1]))
                    massDefct += df.loc[(df["Z"] == int(temp4[2])) & (df["N"] == int(temp4[1])), "MM"].values[0] * int(i[1][1][0])
                    mass2 = next((mass for e, mass in self.composition if e.Z == el2.Z and e.N == el2.N), 0)
                    #print(f"moles1: {self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1)}, moles2: {self.returnSpecificNumberOfMoles(elementt = el2, mass = mass2)}")
                    #print(f"mass1: {mass1}: {temp3}")
                    #print(f"mass2: {mass2}: {temp4}")
                    Y_1 = (self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1) * N_a)/self.volume
                    Y_2 = (self.returnSpecificNumberOfMoles(elementt = el2, mass = mass2) * N_a)/self.volume

                    #print(f"Y_1: {Y_1}")
                    #print(f"Y_2: {Y_2}")
                    #molCm3 = rate * ((self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1) * self.returnSpecificNumberOfMoles(elementt = el2, mass = mass2))/(self.volume ** 2))
                    molCm3 = (Y_1 * Y_2 * rate * self.volume)/(N_a)
                    #logging.info(molCm3)
                    #print(f"molCm3: {molCm3}")
                    #print(f"res: {(Y_1 * Y_2 * rate * self.volume)/(N_a)}")

                    productEls = {}
                    for w in i[2]:
                        _, N11, Z11 = w.split("-")
                        productEls[element(Z = int(Z11), N = int(N11))] = w[0]
                        massDefct -= df.loc[(df["Z"] == int(Z11)) & (df["N"] == int(N11)), "MM"].values[0] * int(w[0])
                    totalChangeMoles = (molCm3) * SIMULATION_TIME_INTERVAL# * self.volume
                    for m, (element1, mass) in enumerate(self.composition):
                        if element1 == el1:
                            am = self.composition[m][1] - totalChangeMoles * self.returnMolarMass(el1)
                            if am > 0:
                                self.composition[m] = (el1, am)
                                #print(f"mole change: {totalChangeMoles}")
                            else:
                                self.composition[m] = (el1, 0)
                        if element1 == el2:
                            am = self.composition[m][1] - totalChangeMoles * self.returnMolarMass(el2)
                            if am > 0:
                                self.composition[m] = (el2, self.composition[m][1] - totalChangeMoles * self.returnMolarMass(el2))
                                #print(f"mole change: {totalChangeMoles}")
                            else:
                                self.composition[m] = (el2, 0)
                        if element1 in productEls:
                            self.composition[m] = (element1, self.composition[m][1] + totalChangeMoles * self.returnMolarMass(element1)*int(productEls[element1]))
                            productEls.pop(element1)
                    for q, ppp in productEls.items():
                        self.composition.append((q, totalChangeMoles * self.returnMolarMass(q) * int(ppp)))
                    self.calculateNumberOfMoles()#This might be too much, and may slow down the operation too much to justify the small amount of extra accuracy. If true, remove it.
                    deltaE = massDefct * mu * c * c
                    self.temperature += (2 * deltaE)/(3 * self.numberOfMoles * R)
                    #print("RAN!")
                    for iiiii in self.composition:
                        print(f"{iiiii[0].Z}-{iiiii[0].N}|{iiiii[1]}")
                else: #if i == 1
                    massDefct = 0.0
                    temp3 = i[1][0].split("-")
                    #temp = i[1][0].split("-")
                    #el1 = element(Z = int(temp[2]), N = int(temp[1]))
                    el1 = element(Z = int(temp3[2]), N = int(temp3[1]))
                    massDefct += df.loc[(df["Z"] == int(temp3[2])) & (df["N"] == int(temp3[1])), "MM"].values[0] * int(i[1][0][0])
                    mass1 = next((mass for e, mass in self.composition if e.Z == el1.Z and e.N == el1.N), 0)
                    #print(f"mass: {mass1}")
                    #print(f"moles1: {self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1)}")
                    Y_1 = (self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1) * N_a)/self.volume

                    #print(f"Y_1: {Y_1}")
                    #molCm3 = (0.5 * rate * ((self.returnSpecificNumberOfMoles(elementt = el1, mass = mass1))/(self.volume)) ** 2)
                    molCm3 = (0.5 * Y_1 * Y_1 * rate * self.volume)/N_a
                    #logging.info(molCm3)
                    #print(f"molCM3: {molCm3}")
                    productEls = {}
                    for w in i[2]:
                        _, N11, Z11 = w.split("-")
                        #                      \/The element                      \/The number of them produced
                        productEls[element(Z = int(Z11), N = int(N11))] = w[0]
                        massDefct += -df.loc[(df["Z"] == int(Z11)) & (df["N"] == int(N11)), "MM"].values[0]

                    totalChangeMoles = (molCm3) * SIMULATION_TIME_INTERVAL# * self.volume
                    for m, (element1, mass) in enumerate(self.composition):
                        if element1 == el1:
                            self.composition[m] = (el1, self.composition[m][1] - totalChangeMoles * self.returnMolarMass(el1))
                            #print(f"mole change: {totalChangeMoles}")
                        if element1 in productEls:
                            self.composition[m] = (element1, self.composition[m][1] + totalChangeMoles * self.returnMolarMass(element1)*int(productEls[element1]))
                            productEls.pop(element1)
                    for q, ppp in productEls.items():
                        self.composition.append((q, totalChangeMoles * self.returnMolarMass(q) * int(ppp)))
                    self.calculateNumberOfMoles() #This might be too much, and may slow down the operation too much to justify the small amount of extra accuracy. If true, remove it.
                    deltaE = massDefct * mu * c * c
                    self.temperature += (2 * deltaE)/(3 * self.numberOfMoles * R)
                    #print("RAN!")
                    for iiiii in self.composition:
                        print(f"{iiiii[0].Z}-{iiiii[0].N}|{iiiii[1]}")
    #log
    #element
    #Instead of calculating the r_0 and r_1 for all layers, instead just calculate r_1 i.e. the upper radius, and fix the lower radius of all layers to be the one below them.
    def gravitationalSettling(self, cumulativeMass: float):
        dConst = ((3*math.pow(k*self.temperature*self.temperature, 1.5))/(16*self.pressure*math.sqrt(2 * pi)))
        g = 0
        if self.radius_0 == 0:
            g = 0
        else:
            g = (G * cumulativeMass) / (self.radius_0**2)
        ratesInfo = [[]]

        if len(self.composition) == 1:
            return []

        for i in range(len(self.composition)):
            iiii = self.composition[i]
            modC = self.composition.pop(i)

            oneOverD = 0.0
            # - self.returnSpecificNumberOfMoles(iiii[0], iiii[1])
            meanMolecular = (self.mass - iiii[1])/((self.numberOfMoles)*N_a)
            for j in modC:
                mole = self.returnSpecificNumberOfMoles(element = j[0], mass = j[1])
                m = (j[0].N*i[0].N)/(j[0].N + iiii[0].N)
                sigmaaa = pi * (1.2*10**(-17)*(i[0].N + j[0].N))**2
                D = dConst/(math.sqrt(m)*sigmaaa**2)
                oneOverD += (mole)/(D)
            v = (1/oneOverD)*((meanMolecular*g)/(k * self.temperature))
            x = self.returnSpecificNumberOfMoles(j[0])/self.numberOfMoles
            n = (self.pressure * x)/(k * self.temperature)
            rateOfMass = n * j[0].N * mu * v * 4 * pi * (((self.radius_0 + self.radius_1)/2)**2) * SIMULATION_TIME_INTERVAL
            ratesInfo.append([iiii[0] ,rateOfMass])

        return ratesInfo

    def predictUpperBoundVelocity(self): #Untested
        self.projectedVelocity = (self.forces*SIMULATION_TIME_INTERVAL)/self.mass + self.velocity

    def changeUpperBoundPosition(self): #Untested
        self.radius_1 += ((self.velocity + self.projectedVelocity) * SIMULATION_TIME_INTERVAL)/2
        self.velocity = self.projectedVelocity
        return self.radius_1

    def correctLowerBound(self, correctedValue: float): #Untested
        self.radius_0 = correctedValue

    def totalEnergy(self) -> dict["massEnergy": int, "kineticEnergy": int, "thermalEnergy": int]:
        return {"massEnergy": self.mass * c * c, "kineticEnergy": 0.5 * self.mass * self.velocity * self.velocity, "thermalEnergy": 1.5 * self.pressure * self.volume}

    def checkPresence(self, nuclei) -> bool:
        for nucleus in nuclei[1]:
            _, N, Z = nucleus.split("-")
            if (int(Z), int(N)) not in self.elementSet:
                return False
        return True

    def returnMolarMass(self, tup: element):
        total = df.loc[(df["Z"] == tup.Z) & (df["N"] == tup.N), "MM"].values[0]/1000
        return total

    def returnSpecificNumberOfMoles(self, elementt: element, mass: decimal.Decimal): # Works (tested C12, C13)
        #print(elementt.Z)
        #print(elementt.N)
        #moles = mass/df.loc[(df["Z"] == elementt.Z) & (df["N"] == elementt.N), "MM"].values[0] * 1000
        #return moles
        matches = df.loc[(df["Z"] == elementt.Z) & (df["N"] == elementt.N), "MM"]
        if matches.empty:
            raise ValueError(f"No molar mass found for Z={elementt.Z}, N={elementt.N}")
        return (mass * 1000)/ matches.values[0]

In [10]:
test = starLayer(composition=[(element(Z = 1, N = 1), 2*10**(29))], temperature=15000000, radius_0=0, radius_1=139000000, velocity=0)

#testElement = element(Z = 6, N = 13)
#e = test.returnSpecificNumberOfMoles(elementt = testElement, mass = 12.1)
#logging.info(e)
#

test.calculatePressure()
#logging.info(pre)
test.calculateNumberOfMoles()


In [11]:
test.fusion()


1-1|2e+29
1-2|285723269478.7261
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
0-1|[0.00013225]
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
0-1|[0.00013225]
1-3|[0.00040221]
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
0-1|[0.00013225]
1-3|[0.00040221]
2-4|[9.7742212e-17]
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
0-1|[0.00013225]
1-3|[0.00040221]
2-4|[2.49464782e-07]
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
0-1|[0.00013225]
1-3|[0.00040221]
2-4|[2.49464782e-07]
3-7|[3.89408383e-46]
1-1|[2.e+29]
1-2|[2.29324254e+11]
2-3|[8.44550586e+10]
0-1|[0.00013225]
1-3|[0.00040221]
2-4|[2.49464782e-07]
3-7|[3.89408383e-46]


In [12]:
#test.fusion()


In [13]:
class Star:
    def __init__(self, starLayers: [starLayer]): #Untested
        self.starLayers = starLayers
        self.starLayersInformation = [[]]# This is to try and model the emission spectrum
        self.outputPower = 0
        self.averagePressure = 0
        self.radius = 0
        self.composition = {}
        self.radiationPressure = 0
        self.reactions = 0

    def timeIncrement(self): #Untested
        self.starLayersInformation = [[]]
        self.outputPower = 0
        self.averagePressure = 0
        self.radius = 0
        self.radiationPressure = 0

        for i in self.starLayers:
            i.startProcedure()

        totalReactions = 0
        #To make life simple, this is where all the time-dependant functions will be run
        #Calculating forces
        self.calculateForcesOnLayersAndRadiativeHeating()
        #Move
        self.move()
        #Adiabatic heating
        for i in self.starLayers:
            i.calcAdibetHeat()
        #Non-radiative heat transfer
            #Not going to do
        #Element separation
        self.elementTransfer()
        #Radiated energy
            #Calculated in calculating forces stage
        #Fusion
        self.doFusion()
        self.generatePressureStatement()
        self.findRadius()
        self.findNumberOfMoles()

        #Logging
        f = open("runtimeInfo", 'a')
        f.write(f"--------")
        f.write(f"Time: {time}")
        f.write(f"Total Reactions: {self.reactions}")
        f.write(f"Average Pressure: {self.averagePressure}")
        f.write(f"Power output: {self.outputPower}")
        f.write(f"Average temperature {(self.pressure * ((4/3)*pi*self.radius**3))/(self.numberOfMoles * R)}")
        f.write(f"Radius of gas cloud {self.radius}")
        f.write(f"Average density {self.mass/((4/3)*pi*self.radius**3)}")
        f.write(f"Radiation pressure {self.radiationPressure}")
        f.write("\n")
        f.write(f"Composition: {self.composition}")
        f.write(f"--------")




    def getElementsList(self):
        for i in self.starLayers:
            itemComp = i.composition
            for j in itemComp:
                if j[0] in self.composition:
                    self.composition[j[0]] += j[1]
                else:
                    self.composition[j[0]] = j[1]


    def generatePressureStatement(self):
        summ = 0.0
        for i in self.starLayers:
            summ += i.pressure * i.volume
        summ = summ/((4/3)*pi*self.radius**3)
        self.averagePressure = summ

    def findRadius(self):
        self.radius = self.starLayers[len(self.starLayers)-1].radius_1

    def findNumberOfMoles(self):
        for i in self.starLayers:
            self.numberOfMoles += i.numberOfMoles

    def doFusion(self): #Untested

        for i in self.starLayers:
            i.fusion()


    def elementTransfer(self):
        ratesMegaSet = [list()]
        cumulativeMass = 0.0



        for i in self.starLayers:
            cumulativeMass += i.mass
            ratesMegaSet.append(i.gravitationalSettling(cumulativeMass=cumulativeMass))
        for i in range(len(ratesMegaSet)):
            if len(ratesMegaSet[i]) == 0:
                continue
            else:
                item = ratesMegaSet[i]
                self.addValues(self.starLayers[i-1].composition, self.starLayers[i].composition, self.starLayers[i+1].composition, item)







    def addValues(A, B, C, tups):
        for tup in tups:
            key, value = tup
            abs_value = abs(value)

            # 1. Update B by subtracting abs(value)
            for i, (k, v) in enumerate(B):
                if k == key:
                    B[i] = (k, v - abs_value)
                    break
            else:
                print(f"Key {key} not found in B.")
                continue  # Skip to the next tuple

            # 2. Depending on sign, update A or C
            target_list = A if value >= 0 else C

            for i, (k, v) in enumerate(target_list):
                if k == key:
                    target_list[i] = (k, v + abs_value)
                    break
            else:
                # If not found, append
                target_list.append((key, abs_value))

    def calculateForcesOnLayersAndRadiativeHeating(self):
        self.starLayersInformation = [[]]
        cumulativeMass = 0.0

        power  = 0.0 #This is the amount of radiation outgoing.

        rangee = range(len(self.starLayers))
        lenn = len(self.starLayers) - 1
        for i in rangee:
            item = self.starLayers[i]
            item.forces = 0
            self.starLayersInformation.append([item.temperature, item.radius_1])
            if i < lenn:
                area0 = item.radius_0 ** 2 * pi * 4
                area1 = item.radius_1 ** 2 * pi * 4
                item.temperature += (2*power*SIMULATION_TIME_INTERVAL*N_a)/(3 * R)
                item.temperature -= (2*(4 * area1 * sigma * item.temperature ** 4)*SIMULATION_TIME_INTERVAL*N_a)/(3 * R)#Accounting for energy loss via radiation
                item.calculatePressure()
                powerAfter = power * math.pow(e, (item.radius_1 - item.radius_0) * 0.4 * (item.mass/item.volume)) # Change in radiation after going through the gas layer

                radiationPressure = (power - powerAfter)/(c)
                self.radiationPressure += (2*radiationPressure)/(area0 + area1) #Radiation pressure
                item.forces += item.pressure * area0 - (cumulativeMass*item.mass*G)/(pi * ((item.radius_0 + item.radius_1)/2)**2) - self.starLayers[i+1].pressure * area1 + radiationPressure * SIMULATION_TIME_INTERVAL
                power = powerAfter
                power += 4 * area1 * sigma * item.temperature ** 4
            else: #Outermost layer
                area0 = item.radius_0 ** 2 * pi * 4
                area1 = item.radius_1 ** 2 * pi * 4
                item.temperature += (2*power*SIMULATION_TIME_INTERVAL*N_a)/(3 * R)
                item.temperature -= (2*(4 * area1 * sigma * item.temperature ** 4)*SIMULATION_TIME_INTERVAL*N_a)/(3 * R)#Accounting for energy loss via radiation
                item.calculatePressure()
                powerAfter = power * math.pow(e, (item.radius_1 - item.radius_0) * 0.4 * (item.mass/item.volume)) # Change in radiation after going through the gas layer

                radiationPressure = (power - powerAfter)/(c)
                self.radiationPressure += (2*radiationPressure)/(area0 + area1) #Radiation pressure
                item.forces += item.pressure * area0 - (cumulativeMass*item.mass*G)/(pi * ((item.radius_0 + item.radius_1)/2)**2) + radiationPressure*SIMULATION_TIME_INTERVAL
                power = powerAfter
                power += 4 * area1 * sigma * item.temperature ** 4
            cumulativeMass += item.mass
        self.outputPower = power

    # The reason why the function determining the movement of the layers is not in the layers model is because the movement is dependent on the other layers as well.
    #\/ this function should only be run once the new velocity has been calculated.
    def move(self): #Untested
        prev = 0.0
        for i in self.starLayers:
            #todo
            i.correctLowerBound(correctedValue = prev)
            i.predictUpperBoundVelocity()
            prev = i.changeUpperBoundPosition()





# PP chain, CNO cycles and Helium capture
To model the actual fusion of the particles themselves, and to calculate what products they would form would be much too complicated. Because of this, I will predefine certain reactions that are allowed to happen in my star model.

I won't, however, bother to add the hot CNO cycles (for now), since they typically only occur duing supernovae, and that is far beyond the scope of this model.

Around 2~3% of the energy output of the sun is via neutrinos. I'm not going to model this, since it would make life much more difficult for very little reward

## PP-I Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^3_2He \rightarrow ^4_2He + 2^1_1H$$

---
## PP-II Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^4_2He \rightarrow ^7_4be + \gamma$$

$$^7_4Be + ^0_0e^- \rightarrow ^7_3Li + v_e$$

$$^7_3Li + ^1_1H \rightarrow 2^4_2He$$


---
## PP-III Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^4_2He \rightarrow ^7_4He + \gamma$$

$$^7_4Be + ^1_1H \rightarrow ^8_5B + \gamma$$

$$^8_5B \rightarrow ^8_4Be + ^0_0e^+ + v_e$$

$$^8_4Be \rightarrow 2^4_2He$$

In the rates data, the last two reactions are listed as one, this is probably because its half life is ~0.77s.

---
## PP-IV Chain

$$^1_1H + ^1_1H \rightarrow ^2_2He + \beta^+ + v_e$$

$$^2_2He + ^1_1H \rightarrow ^3_2He + \gamma$$

$$^3_2He + ^1_1H \rightarrow ^4_2He + ^0_0e^+ + v_e$$


---

## Helium capture

$$^4_2He + ^4_2He \rightarrow ^8_4Be$$
$$^4_2He + ^8_4Be\rightarrow ^{12}_6C$$
$$^4_2He + ^{12}_6C\rightarrow ^{16}_8O$$
$$^4_2He + ^{16}_8O\rightarrow ^{20}_10Ne$$
$$^4_2He + ^{20}_10Ne\rightarrow ^{24}_12Mg$$
$$...$$

This occurs all the way up to $^{56}_{28}Fe$, and then stops due to binding energy going down again.

---

### **CNO-I Cycle (Main Branch)**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$


---

### **CNO-II Cycle**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$

$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$


---

### **CNO-III Cycle**



$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$


---

### **CNO-IV Cycle**


$${}^{12}\text{C} + p \rightarrow {}^{13}\text{N} + \gamma$$

$${}^{13}\text{N} \rightarrow {}^{13}\text{C} + e^+ + \nu_e$$

$${}^{13}\text{C} + p \rightarrow {}^{14}\text{N} + \gamma$$

$${}^{14}\text{N} + p \rightarrow {}^{15}\text{O} + \gamma$$

$${}^{15}\text{O} \rightarrow {}^{15}\text{N} + e^+ + \nu_e$$

$${}^{15}\text{N} + p \rightarrow {}^{12}\text{C} + {}^4\text{He}$$





# Running

In [14]:
mass = 4000000000000000000000000000000.0 #float(input("Mass of hydrogen cloud: "))
radius = 60000000000000000000.0 #float(input("Radius of hydrogen cloud: "))
numberOfLayers = 2000 #int(input("Number of layers: "))
temperature = 5.0 #float(input("Temperature of hydrogen cloud: "))

SIMULATION_TIME_INTERVAL = 131000000

#comp = []
#radi = 0.0
#incr = radius/numberOfLayers
#for i in range(numberOfLayers):
 #   masss = (mass * ((radi+incr)**3 - radi**3))/(radius ** 3)
 #   comp.append(starLayer(composition=[(element(Z=1,N=1), masss)], temperature=temperature, radius_0=radi, radius_1=radi+incr, velocity = 0))
  #  radi += incr
#star = Star(starLayers=comp)
#i = input()
#star.timeIncrement()
#i=input()

In [15]:
test1 = starLayer(composition=[(element(Z = 1, N = 1), 2*10**(29))], temperature=15000000, radius_0=0, radius_1=139000000, velocity=0)

#testElement = element(Z = 6, N = 13)
#e = test.returnSpecificNumberOfMoles(elementt = testElement, mass = 12.1)
#logging.info(e)
#

test1.calculatePressure()
#logging.info(pre)
test1.calculateNumberOfMoles()


In [16]:
test1.fusion()

1-1|1.999999999812707e+29
1-2|3.742974830171312e+19
1-1|[1.99515696e+29]
1-2|0
2-3|[1.44933326e+27]
1-1|[1.99515696e+29]
1-2|[0.]
2-3|[1.44933326e+27]
0-1|[0.]
1-1|[1.99515696e+29]
1-2|[0.]
2-3|[1.44933326e+27]
0-1|[0.]
1-3|[0.]
1-1|[1.99515696e+29]
1-2|0
2-3|[1.44933326e+27]
0-1|[0.]
1-3|0
2-4|[0.]
1-1|[1.99515696e+29]
1-2|0
2-3|[1.44933326e+27]
0-1|[0.]
1-3|0
2-4|[0.]
1-1|[1.99515696e+29]
1-2|0
2-3|[1.44933326e+27]
0-1|[0.]
1-3|0
2-4|0
3-7|[0.]
1-1|[1.99515696e+29]
1-2|0
2-3|[1.44933326e+27]
0-1|0
1-3|[0.]
2-4|0
3-7|[0.]


# Sources used:

The sources are *not* given in chronological order of access.

1. https://web.archive.org/web/20170115214447/https://zuserver2.star.ucl.ac.uk/~idh/PHAS2112/Lectures/Current/Part7.pdf

2. https://en.wikipedia.org/wiki/Stellar_nucleosynthesis#cite_note-40

3. https://www.youtube.com/@DrPhysicsA (All of nuclear playlist)

4. https://t2.lanl.gov/nis/data/astro/ for S-Factor experimental data

5. https://reaclib.jinaweb.org/popularRates.php for rates data

6. https://en.wikipedia.org/wiki/Adiabatic_process Adiabetic heating

7. https://chatgpt.com/ convection, gravitational settling



# How AI was used in this project

ChatGPT was used in the generation of the LaTEX displaying the CNO-I through to CNO-IV cycle.

ChatGPT was used to learn the gravitational settling and convection equations.

Deepseek was used to write the code to parse data from Reaclib1 files.

ChatGPT was used in *no other aspect of this project*.